# 分类评估模板：该模板可以对主动、被动交易进行分类统计
---
    时间：0903
    文件：严苛三买.py
    策略：使用不同的K线组合计算中枢和笔，再计算第三买卖点；进入新的中枢平仓，回到所在中枢止损；使用平台数据进行回测；
    调用：market_timing_lab_v0827;three_point_class_v0827
    备注：在回测进行的过程中，只要已经回测完一个合约就可以使用【分类评估模板】随时调用中间结果，并做评估。！！请注意参数匹配。

In [1]:
import os 
os.chdir('/home/jovyan/xhq/main/parting/')
from backtest_class import backtest,classify
import numpy as np
import pandas as pd
from ResAssese import ResultAssese
from algoqi.data import D, plotutils
D.switch_zxzx_client(access_key='e7e2de63f95674632824ed6b2be8dd1f', secret_key='d43ca617fe7048e4ccfb158dde8e1c32')
D.load()
from three_point_class_v0827 import three_point,three_point_backtest
from highcharts import Highstock

#实现一个cell多个输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 1 参数

In [2]:
start = '20190102' #回测开始时间（必须是交易日）
end = '20191220' #回测结束时间（建议使用某个当月合约的结束时间）

k_min_high_level = 1  #int：计算中枢的K线级别，如k_min_high_level =30 表示计算中枢的高级K线是30min K线
k_min_low_level = 1  #int：计算笔的K线级别
use_tick_merge = False  #use_tick_merge = True，使用tick数据合成K线；use_tick_merge = False，使用平台提供的K线数据。
py_path0 = './第三买卖点/最终回测/严苛三买.py' #策略文件路径
raw_save_name = '严苛三买' #str：建议使用策略文件名

date_time = '0903'  #str: 回测当日日期，建议作为尾注，放入save_name0中，以免文件文件同名覆盖


if use_tick_merge：
    flag = 'use_tick' + date_time
else:
    flag = 'use_lab' + date_time

save_name0 = f'{k_min_high_level}_{k_min_low_level}min_{raw_save_name}_{flag}_' #最终合成的名字

### 2 综合统计

In [3]:
# #读取中间成果、获取原始交易记录trades_df0和日志记录trades_df0
# trades_df0 = pd.read_pickle('./第三买卖点/strategy_save/trades_'+save_name0+start+'_'+end+'.pkl')
# trades_df0 = pd.read_pickle('./第三买卖点/strategy_save/logs_'+save_name0+start+'_'+end+'.pkl')

#### 说明
---
    ret_long: 多仓每笔收益统计分析
    ret_short: 空仓每笔收益统计分析
    ret_all: 多头仓位结合，对每笔收益进行统计分析
    count: 交易次数（同一笔交易，开仓平仓算作一次）
    trades_count: 交易次数（同一笔交易，开仓平仓各算作一次，反映交易手续费）
    count_ratio: 交易次数占比
        
    OS: 开多仓
    OL: 开空仓
    CL(NLC): 满足策略主动平仓条件，平多仓
    CS(NSC): 满足策略主动平仓条件，主动平空仓
    破底CL(SLC): 触发止损条件，平多仓
    破顶CS(SSC): 触发止损条件，平空仓
    end_CS(ELC): 触发结算平仓条件，平多仓
    end_CL(ESC)： 触发结算平仓条件，平空仓
    
        
    注：
    1. O-open C-close S-short L-long N-natural（主动） S-stop（止损） 
    2. NLC、NSC、SLC、SSC、ELC、ESC表示在图中的标签名
    3. 主动收益:CL&CS ,结算收益：end_CL&end_CS,止损收益：破底CL&破底CS
    4. average_ret:平均每笔收益、ProfitCossRatio：盈亏比、ProfitRate：胜率、trades_count：总计交易次数、average_pos_time：平均持仓时间
    5. 图下方控件可以控制该类图元的显示
    6. classify.classify_analysis函数中，classify_name: active-主动交易；passive-止损交易；end-结算交易
    

In [4]:
analysis_close0,analysis_close_res0,long_res_dict0,long_fig_dict0 = three_point_backtest.analysis(save_name0,start,end)
analysis_close0
print('\n 主动收益:CL&CS ,结算收益：end_CL&end_CS,止损收益：破底CL&破底CS\n')
analysis_close_res0
print('***************'*5)
print('average_ret:平均每笔收益、ProfitCossRatio：盈亏比、ProfitRate：胜率、trades_count：总计交易次数、average_pos_time：平均持仓时间')
long_res_dict0['ResultAssese']

开多头次数是否等于平多头次数： True
开空头次数是否等于平空头次数： True
是否只有平掉现有的头寸，才能开新的仓位： True
***************************************************************************


ret_long                                   ret_short              \
                 amax   amin   std   mean median    sum      amax  amin   std   
logs_shorter                                                                    
CL              335.6  335.6   NaN  335.6  335.6  335.6       0.0   0.0   NaN   
破底CL             -5.4  -32.2  11.2  -18.7  -17.4 -130.6       0.0   0.0   0.0   
破顶CS              0.0    0.0   0.0    0.0    0.0    0.0     -14.2 -48.2  15.7   
Row_sum           NaN    NaN   NaN    NaN    NaN  205.0       NaN   NaN   NaN   

                                 ret_all                                    \
              mean median    sum    amax   amin   std   mean median    sum   
logs_shorter                                                                 
CL             0.0    0.0    0.0   335.6  335.6   NaN  335.6  335.6  335.6   
破底CL           0.0    0.0    0.0    -5.4  -32.2  11.2  -18.7  -17.4 -130.6   
破顶CS         -37.5  -43.7 -149.8   -14.2  -48.2  15.7  -37.5  -43.7 -149.8   
Row_sum        NaN    NaN -149.8     NaN    NaN   NaN    NaN    NaN   55.2   

             count count_ratio  
                                
logs_shorter                    
CL             1.0      0.0833  
破底CL           7.0      0.5833  
破顶CS           4.0      0.3333  
Row_sum       12.0      1.0000


 主动收益:CL&CS ,结算收益：end_CL&end_CS,止损收益：破底CL&破底CS



,count,count_ratio,min,25%,50%,75%,max,std,mean,return_sum
logs_classify,,,,,,,,,,
主动收益,1.0,0.0833,335.6,335.6,335.6,335.6,335.6,NaN,335.6,335.6
止损收益,11.0,0.9167,-48.2,-37.2,-25.0,-13.2,-5.4,15.5,-25.5,-280.4
Row_sum,12.0,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.2


***************************************************************************
average_ret:平均每笔收益、ProfitCossRatio：盈亏比、ProfitRate：胜率、trades_count：总计交易次数、average_pos_time：平均持仓时间


,average_ret,ProfitCossRatio,ProfitRate,trades_count,average_pos_time
long,25.625,2.569678,0.125000,16,1 days 12:34:00
short,-37.450,0.000000,0.000000,8,0 days 11:31:00
total,4.600,1.196862,0.083333,24,1 days 04:13:00


In [5]:
#单笔收益柱状图：时间戳是该笔交易的平仓时间
long_fig_dict0['FigRetPerSize_update']

In [6]:
# 累计收益曲线（pnl线）
long_fig_dict0['FigAccuRet']

In [7]:
# 获取综合trades和logs
trades_df,logs_df = three_point_backtest.read_df(save_name0,start,end)

### 3 主动交易统计
---
    仅对主动交易进行统计

In [8]:
active_res_dict,active_fig_dict= classify.classify_analysis(trades_df,logs_df,start,end,classify_name = 'active')
active_res_dict['ResultAssese']

,average_ret,ProfitCossRatio,ProfitRate,trades_count,average_pos_time
long,335.6,-inf,1.0,2,11 days 00:26:00
short,NaN,NaN,NaN,0,NaT
total,335.6,-inf,1.0,2,11 days 00:26:00


In [9]:
#加入平仓标识的每笔收益,并且每笔收益的时间戳都是平仓时间
active_fig_dict['FigRetPerSize_update']

In [10]:
active_fig_dict['FigAccuRet']

### 4 止损交易统计
---
    仅对止损交易进行统计

In [11]:
passive_res_dict,passive_fig_dict= classify.classify_analysis(trades_df,logs_df,start,end,classify_name = 'passive')
passive_res_dict['ResultAssese']

,average_ret,ProfitCossRatio,ProfitRate,trades_count,average_pos_time
long,-18.657143,0.0,0.0,14,04:01:00
short,-37.450000,0.0,0.0,8,11:31:00
total,-25.490909,0.0,0.0,22,06:44:00


In [12]:
#加入平仓标识的每笔收益,并且每笔收益的时间戳都是平仓时间
passive_fig_dict['FigRetPerSize_update']

In [13]:
passive_fig_dict['FigAccuRet']

### 绘图

#### 说明
---
    level2: level2 = False 表示k_min_high_level不等于k_min_low_level，图中需要额外绘制高级K线的笔，否则反之。
    save_html: save_html = True 表示保存生成的html图，保存路径为./第三买卖点/draw/trading_point_{save_name0}_{start}_{end}_{flag}
    flag: 为html图名称加一个尾注，建议使用绘制日期，如0902

    图中：
    IC00.CFE prices: 低级K线
    trading_point_L3(L3):第三类买点
    trading_point_S3(S3): 第三类卖点
    trend_price_low_level: 低级K线的笔
    trend_price_high_level: 高级K线的笔
    pivot_high_level: 根据高级K线计算的中枢
    pivot_time: 中枢确立时间
    

In [14]:
param0 = {
        'start':start,
        'end' :end,
        'py_path0' :py_path0,
        'save_name0':save_name0,
        'k_min_high_level':k_min_high_level,
        'k_min_low_level':k_min_low_level,
        'code':"IC00.CFE",
        'use_tick_merge':use_tick_merge}

H0 = three_point_backtest.draw_v0903(param0,level2 = True,save_html = False,flag = date_time)

In [15]:
H0